# Requirements → User Stories → Test Cases (LLM-powered)

This notebook:

- Ingests a requirements document (`.txt`, `.docx`, or `.pdf`)
- Uses a Large Language Model (LLM) to:
  - Extract EPICs and user stories
  - Generate test cases from those user stories
- Produces a combined Excel file with columns:

  - `EPIC`
  - `User Story`
  - `Test Case Name`
  - `Test Case Description`
  - `Execution Steps`

It also exposes the exact prompts used so they can be discussed and evaluated.

In [1]:
import os
import json
from typing import List, Dict, Any

import pandas as pd
import docx
import pdfplumber

from openai import OpenAI

In [2]:
# Set your OpenAI API key as an environment variable before running:
# On macOS/Linux:
#   export OPENAI_API_KEY="sk-..."
# On Windows (PowerShell):
#   setx OPENAI_API_KEY "sk-..."

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable before running this notebook.")

client = OpenAI(api_key=OPENAI_API_KEY)

# Default model (change if needed, for example to "gpt-4.1" or "gpt-4o")
DEFAULT_MODEL = "gpt-4.1-mini"

In [3]:
def call_llm(prompt: str, model: str = DEFAULT_MODEL, temperature: float = 0.2) -> str:
    """
    Helper to call the LLM with a simple system + user message.
    Returns the text content of the first choice.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert business analyst and QA engineer. "
                    "You always follow the instructions exactly and return valid JSON when asked."
                ),
            },
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content.strip()

## Document Loading Utilities

These helpers load requirements from `.txt`, `.docx`, or `.pdf` into a single text string.

In [4]:
def load_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()


def load_docx(path: str) -> str:
    doc = docx.Document(path)
    return "\n".join(p.text for p in doc.paragraphs if p.text.strip())


def load_pdf(path: str) -> str:
    text_chunks = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text_chunks.append(page_text)
    return "\n".join(text_chunks)


def load_requirement_file(path: str) -> str:
    """
    Load requirement text from .txt, .docx, or .pdf.
    """
    path_lower = path.lower()
    if path_lower.endswith(".txt"):
        text = load_txt(path)
    elif path_lower.endswith(".docx"):
        text = load_docx(path)
    elif path_lower.endswith(".pdf"):
        text = load_pdf(path)
    else:
        raise ValueError("Unsupported file type. Use .txt, .docx, or .pdf")

    # Basic cleanup
    text = text.replace("\r", "\n")
    text = "\n".join(line.strip() for line in text.splitlines() if line.strip())
    return text

In [5]:
# Set your requirement file path here.
# Example:
# requirement_path = "../data/sample_requirement.docx"
# requirement_path = "../data/sample_requirement.pdf"
# requirement_path = "../data/sample_requirement.txt"

requirement_path = "../data/sample_requirement_2.pdf"  # <-- change to your file

requirements_text = load_requirement_file(requirement_path)

print("Loaded requirement file:", requirement_path)
print("Number of characters:", len(requirements_text))
print("\nPreview (first 1000 characters):\n")
print(requirements_text[:1000])

Loaded requirement file: ../data/sample_requirement_2.pdf
Number of characters: 72381

Preview (first 1000 characters):

SOFTWARE
REQUIREMENTS
SPECIFICATION
Hourly Student Appointments
04/11/2011
The new Hourly Student Appointment System will fully automate the
processing of new positions, changes and terminations of positions for
hourly paid MIT students.
Version 2
Prepared by: Jyoti Sharma
Review and edits by: Project Team, Business owners providing sign-off
HSA Software Requirements Specification
INTRODUCTION ........................................................................................................................................... 3
Purpose .................................................................................................................................................................................... 3
Intended Audience and Reading Suggestions ...........................................................................................................

## LLM Prompt: EPIC and User Story Extraction

The following prompt is used to extract EPICs and user stories from the requirements document.
It instructs the model to return **only valid JSON** in a specific schema.

In [6]:
USER_STORIES_PROMPT_TEMPLATE = """
You are an expert business analyst.

You will be given a software requirements document. Your task is to:
1. Identify high-level EPICs.
2. For each EPIC, write clear agile-style user stories with acceptance criteria.

Return the result as valid JSON ONLY, with this exact structure:

[
  {{
    "epic_name": "string",
    "epic_description": "string",
    "user_stories": [
      {{
        "id": "US-1",
        "title": "string",
        "as_a": "string",
        "i_want": "string",
        "so_that": "string",
        "acceptance_criteria": [
          "criterion 1",
          "criterion 2"
        ]
      }}
    ]
  }}
]

Important rules:
- Output must be valid JSON. No comments, no extra keys, no trailing commas.
- Do NOT include any explanation or text outside the JSON.
- EPIC names and user story titles should be short but descriptive.
- Acceptance criteria must be testable and clear.

Requirements document:
----------------------
{requirements_text}
"""

In [7]:
def generate_user_stories(requirements_text: str) -> List[Dict[str, Any]]:
    """
    Call the LLM to generate EPICs and user stories from the requirements text.
    Returns a list of EPIC dictionaries.
    """
    prompt = USER_STORIES_PROMPT_TEMPLATE.format(requirements_text=requirements_text)
    raw_output = call_llm(prompt)

    # Try to parse as JSON
    try:
        data = json.loads(raw_output)
    except json.JSONDecodeError as e:
        print("Failed to parse LLM output as JSON.")
        print("Raw output:")
        print(raw_output)
        raise e

    if not isinstance(data, list):
        raise ValueError("Expected a list of EPICs in JSON output.")

    return data

In [8]:
epics_with_stories = generate_user_stories(requirements_text)

print(f"Number of EPICs generated: {len(epics_with_stories)}")
if epics_with_stories:
    print("First EPIC (raw JSON):")
    print(json.dumps(epics_with_stories[0], indent=2))

Number of EPICs generated: 6
First EPIC (raw JSON):
{
  "epic_name": "Student Eligibility Verification",
  "epic_description": "Verify that a student is eligible to be hired on the hourly payroll before creating a position.",
  "user_stories": [
    {
      "id": "US-1",
      "title": "Verify student eligibility",
      "as_a": "Student Administrator",
      "i_want": "to verify that a student meets all eligibility criteria for hourly payroll",
      "so_that": "I can proceed with creating a new position only for eligible students",
      "acceptance_criteria": [
        "System allows search for student by unique identifier",
        "System verifies student eligibility based on enrollment status, I-9 status, and residency",
        "System displays student details including name, course, year, degree, email, enrollment and I-9 status",
        "System prevents proceeding if student is non-resident doctoral or special student",
        "System warns if no social security number is on

## LLM Prompt: Test Case Generation

Next, we feed the EPICs and user stories into another prompt that asks the LLM
to generate detailed test cases for each user story.

Again, we ask for strict JSON with a specific schema.

In [9]:
TEST_CASES_PROMPT_TEMPLATE = """
You are an expert QA engineer.

You will be given a list of EPICs and user stories in JSON format.
For EACH user story, generate a set of test cases.

Return the result as valid JSON ONLY, with this structure:

[
  {{
    "epic_name": "string",
    "user_story_id": "US-1",
    "user_story_title": "string",
    "test_cases": [
      {{
        "test_case_name": "string",
        "test_case_description": "string",
        "execution_steps": [
          "step 1",
          "step 2"
        ]
      }}
    ]
  }}
]

Important rules:
- The JSON must be valid. No comments, no extra keys, no trailing commas.
- Do NOT include any explanation or text outside the JSON.
- For each user story, create several test cases that cover main flows and key edge cases.
- Execution steps must be clear, ordered actions that a tester can follow.

EPICs and user stories JSON:
----------------------------
{user_stories_json}
"""

In [10]:
def generate_test_cases(epics_with_stories: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Call the LLM to generate test cases from EPICs and user stories.
    Returns a list of dictionaries, each representing a user story and its test cases.
    """
    user_stories_json = json.dumps(epics_with_stories, indent=2)
    prompt = TEST_CASES_PROMPT_TEMPLATE.format(user_stories_json=user_stories_json)
    raw_output = call_llm(prompt)

    try:
        data = json.loads(raw_output)
    except json.JSONDecodeError as e:
        print("Failed to parse LLM output as JSON.")
        print("Raw output:")
        print(raw_output)
        raise e

    if not isinstance(data, list):
        raise ValueError("Expected a list of test case entries in JSON output.")

    return data


test_case_data = generate_test_cases(epics_with_stories)

print(f"Number of user stories with test cases: {len(test_case_data)}")
if test_case_data:
    print("First user story with test cases (raw JSON):")
    print(json.dumps(test_case_data[0], indent=2))

Number of user stories with test cases: 7
First user story with test cases (raw JSON):
{
  "epic_name": "Student Eligibility Verification",
  "user_story_id": "US-1",
  "user_story_title": "Verify student eligibility",
  "test_cases": [
    {
      "test_case_name": "Search student by unique identifier",
      "test_case_description": "Verify that the system allows searching for a student using their unique identifier.",
      "execution_steps": [
        "Navigate to the student eligibility verification screen.",
        "Enter a valid unique student identifier in the search field.",
        "Click the search button.",
        "Verify that the system displays the student's details."
      ]
    },
    {
      "test_case_name": "Verify eligibility for enrolled student with valid I-9 and residency",
      "test_case_description": "Verify that the system correctly identifies a student as eligible when enrollment status, I-9 status, and residency criteria are met.",
      "execution_steps

## Build Final DataFrame and Export to Excel

The assignment requires an Excel output with the columns:

- `EPIC`
- `User Story`
- `Test Case Name`
- `Test Case Description`
- `Execution Steps`

In [11]:
def build_dataframe(test_case_data: List[Dict[str, Any]]) -> pd.DataFrame:
    """
    Flatten the structured test case data into a tabular format:

    Columns:
    - EPIC
    - User Story
    - Test Case Name
    - Test Case Description
    - Execution Steps
    """
    rows = []

    for item in test_case_data:
        epic_name = item.get("epic_name", "").strip()
        user_story_id = item.get("user_story_id", "").strip()
        user_story_title = item.get("user_story_title", "").strip()

        if user_story_id and user_story_title:
            user_story_display = f"{user_story_id}: {user_story_title}"
        elif user_story_title:
            user_story_display = user_story_title
        else:
            user_story_display = user_story_id

        for tc in item.get("test_cases", []):
            name = tc.get("test_case_name", "").strip()
            desc = tc.get("test_case_description", "").strip()
            steps_list = tc.get("execution_steps", []) or []

            # Join steps into numbered string, one per line
            steps_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(steps_list))

            rows.append(
                {
                    "EPIC": epic_name,
                    "User Story": user_story_display,
                    "Test Case Name": name,
                    "Test Case Description": desc,
                    "Execution Steps": steps_str,
                }
            )

    df = pd.DataFrame(rows, columns=[
        "EPIC",
        "User Story",
        "Test Case Name",
        "Test Case Description",
        "Execution Steps",
    ])
    return df


df = build_dataframe(test_case_data)

print("Number of test case rows:", len(df))
df.head()

Number of test case rows: 47


,EPIC,User Story,Test Case Name,Test Case Description,Execution Steps
0,Student Eligibility Verification,US-1: Verify student eligibility,Search student by unique identifier,Verify that the system allows searching for a ...,1. Navigate to the student eligibility verific...
1,Student Eligibility Verification,US-1: Verify student eligibility,Verify eligibility for enrolled student with v...,Verify that the system correctly identifies a ...,1. Search for a student with active enrollment...
2,Student Eligibility Verification,US-1: Verify student eligibility,Prevent proceeding for non-resident doctoral o...,Verify that the system prevents proceeding if ...,1. Search for a student identified as non-resi...
3,Student Eligibility Verification,US-1: Verify student eligibility,Display student details including enrollment a...,Verify that the system displays all required s...,1. Search for a valid student.\n2. Verify that...
4,Student Eligibility Verification,US-1: Verify student eligibility,Warn and notify if no social security number o...,Verify that the system warns the user and send...,1. Search for a student without a social secur...


In [12]:
# Ensure outputs directory exists
os.makedirs("../outputs", exist_ok=True)

output_path = "../outputs/userstories_testcases_output.xlsx"
df.to_excel(output_path, index=False)

print("Excel file written to:", output_path)

Excel file written to: ../outputs/userstories_testcases_output.xlsx


## Recap and Notes

Pipeline implemented in this notebook:

1. **Document ingestion**
   - Supports `.txt`, `.docx`, and `.pdf`
   - Loads and cleans requirement text

2. **EPIC and user story generation**
   - Prompt: `USER_STORIES_PROMPT_TEMPLATE`
   - Output: strict JSON list of EPICs with user stories and acceptance criteria

3. **Test case generation**
   - Prompt: `TEST_CASES_PROMPT_TEMPLATE`
   - Input: JSON of EPICs and user stories
   - Output: strict JSON mapping each user story to multiple test cases

4. **Tabular output**
   - Flattened into a `pandas` DataFrame with columns:

     - `EPIC`
     - `User Story`
     - `Test Case Name`
     - `Test Case Description`
     - `Execution Steps`

   - Exported to `../outputs/userstories_testcases_output.xlsx`

The two prompt templates are visible in the notebook and can be discussed as part of the "advanced prompt engineering" component of the assignment.